In [1]:
!pip install supervision[assets]

  Obtaining dependency information for supervision[assets] from https://files.pythonhosted.org/packages/50/1a/971a415d67011854c2c59716c49b3de151d54b422c6dd0a6489e0152b42e/supervision-0.20.0-py3-none-any.whl.metadata
  Obtaining dependency information for opencv-python-headless>=4.5.5.64 from https://files.pythonhosted.org/packages/20/44/458a0a135866f5e08266566b32ad9a182a7a059a894effe6c41a9c841ff1/opencv_python_headless-4.9.0.80-cp37-abi3-win_amd64.whl.metadata
   ---------------------------------------- 0.0/38.5 MB ? eta -:--:--
   ---------------------------------------- 0.1/38.5 MB 2.0 MB/s eta 0:00:20
    --------------------------------------- 0.5/38.5 MB 5.3 MB/s eta 0:00:08
   - -------------------------------------- 1.2/38.5 MB 8.5 MB/s eta 0:00:05
   - -------------------------------------- 1.6/38.5 MB 8.6 MB/s eta 0:00:05
   -- ------------------------------------- 2.4/38.5 MB 10.3 MB/s eta 0:00:04
   --- ------------------------------------ 3.5/38.5 MB 12.4 MB/s eta 0:00:03
 

DEPRECATION: Loading egg at c:\users\wen\anaconda3\lib\site-packages\huggingface_hub-0.23.0-py3.8.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..


In [17]:
import torch
from supervision.assets import download_assets, VideoAssets
import cv2
from bytetrack import BYTETracker


ModuleNotFoundError: No module named 'bytetrack'

In [4]:
import sys
sys.path.append('/path/to/ByteTrack')


In [7]:
import bytetrack
print("ByteTrack is successfully imported!")


ModuleNotFoundError: No module named 'bytetrack'

# Download a video

https://supervision.roboflow.com/assets/

In [2]:
download_assets(VideoAssets.VEHICLES)
"vehicles.mp4"

  0%|          | 0/35345757 [00:00<?, ?it/s]

'vehicles.mp4'

In [1]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

Using cache found in C:\Users\Wen/.cache\torch\hub\ultralytics_yolov5_master
c:\Users\Wen\AppData\Local\Programs\Python\Python38\lib\site-packages\transformers\utils\generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
YOLOv5  2024-5-6 Python-3.8.0 torch-2.3.0+cpu CPU

100%|██████████| 14.1M/14.1M [00:00<00:00, 15.0MB/s]

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


In [2]:
cap = cv2.VideoCapture('vehicles.mp4')

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    results = model(frame)

    cv_frame = results.render()[0]

    cv2.imshow('YOLOv5 Detection', cv_frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):  # Press 'q' to exit
        break

cap.release()
cv2.destroyAllWindows()



Integrating ByteTrack for multi-object tracking using the detections from Yolo

In [8]:
import sys
import cv2
import torch

# Assuming ByteTrack and YOLO are correctly installed and configured
sys.path.append('/path/to/ByteTrack')
from bytetrack import BYTETracker

# Load YOLO model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

# Initialize ByteTrack
tracker = BYTETracker()

# Open the video
cap = cv2.VideoCapture('vehicles.mp4')

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Detect objects using YOLO
    results = model(frame)
    detections = results.xyxy[0]  # detections in xyxy format

    # Format detections for ByteTrack (x1, y1, x2, y2, score, class)
    online_targets = tracker.update(detections[:, :4], detections[:, 4], detections[:, 5])

    # Draw tracked objects
    for t in online_targets:
        bbox = t.tlwh
        cv2.rectangle(frame, (int(bbox[0]), int(bbox[1])), (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3])), (0, 255, 0), 2)
        cv2.putText(frame, f'ID: {t.track_id}', (int(bbox[0]), int(bbox[1] - 10)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    cv2.imshow('Frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


ModuleNotFoundError: No module named 'bytetrack'